## LSTM for Gait Phase Estimation

The purpose of this notebook is to train LSTM models to estimate a user's gait phase on both legs, independent of the locomotion mode.

In [1]:
from lstm_modules import *
from convolutional_nn import *
import numpy as np
import tensorflow as tf
import itertools
import pandas as pd

In [2]:
# Seed Random Number Generators

### Tune Hyperparameters

In [3]:
hyperparam_space = {
    'window_size': [20],
    'lstm': {
      'units': [30],
      'activation': ['relu', 'tanh'],
    },
    'dense': {
        'activation': ['tanh']
    },
    'optimizer': {
        'loss': ['mean_squared_error'],
        'optimizer': ['rmsprop']
    }
}

In [4]:
def get_model_configs(hyperparam_space):
    model_configs = []
    for window_size in hyperparam_space['window_size']:
        lstm_params = list(hyperparam_space['lstm'].keys())

        lstm_possibilities = []

        for param in lstm_params:
            lstm_possibilities.append(hyperparam_space['lstm'][param])

        lstm_config_tuples = itertools.product(*lstm_possibilities)
        lstm_configs = []
        for config in lstm_config_tuples:
            lstm_config = {}
            for i, value in enumerate(config):
                lstm_config[lstm_params[i]] = value
            lstm_configs.append(lstm_config)

        dense_params = list(hyperparam_space['dense'].keys())

        dense_possibilities = []

        for param in dense_params:
            dense_possibilities.append(hyperparam_space['dense'][param])

        dense_config_tuples = itertools.product(*dense_possibilities)
        dense_configs = []
        for config in dense_config_tuples:
            dense_config = {}
            for i, value in enumerate(config):
                dense_config[dense_params[i]] = value
            dense_configs.append(dense_config)

        optim_params = list(hyperparam_space['optimizer'].keys())

        optim_possibilities = []

        for param in optim_params:
            optim_possibilities.append(hyperparam_space['optimizer'][param])

        optim_config_tuples = itertools.product(*optim_possibilities)
        optim_configs = []
        for config in optim_config_tuples:
            optim_config = {}
            for i, value in enumerate(config):
                optim_config[optim_params[i]] = value
            optim_configs.append(optim_config)

        possible_configs = itertools.product(lstm_configs, dense_configs, optim_configs)
        config_count = 0
        for config in possible_configs:
            config_count += 1
            model_configs.append({
                'window_size': window_size,
                'lstm': config[0],
                'dense': config[1],
                'optimizer': config[2]
            })
    return model_configs            

In [5]:
hyperparameter_configs = get_model_configs(hyperparam_space)
print(len(hyperparameter_configs))

2


In [6]:
results = []
for model_config in hyperparameter_configs:
    current_result = {}
    current_result['model_config'] = model_config
    current_result['left_validation_rmse'] = []
    current_result['right_validation_rmse'] = []
    for trial in np.arange(1,3):
        dataset = cnn_train_test_split(trial, model_config['window_size'])
        model = lstm_model(sequence_length=model_config['window_size'],
                          n_features=10, 
                           lstm_config=model_config['lstm'],
                           dense_config=model_config['dense'],
                           optim_config=model_config['optimizer'],
                           X_train=dataset['X_train'])
        model.summary()
        early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0)
        model_hist = model.fit(dataset['X_train'], dataset['y_train'], epochs=2, batch_size=128, verbose=1, validation_split=0.2, shuffle=True, callbacks= [early_stopping_callback])
        # model.save('test_rnn_model')

        predictions = model.predict(dataset['X_test'])
        left_rmse, right_rmse, gp = custom_rmse(dataset['y_test'], predictions)

        current_result['left_validation_rmse'].append(left_rmse)
        current_result['right_validation_rmse'].append(right_rmse)
    results.append(current_result)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 20, 10)            21        
_________________________________________________________________
lstm (LSTM)                  (None, 30)                4920      
_________________________________________________________________
dense (Dense)                (None, 4)                 124       
Total params: 5,065
Trainable params: 5,044
Non-trainable params: 21
_________________________________________________________________
Train on 28216 samples, validate on 7055 samples
Epoch 1/2
28216/28216 [==============================] - 7s 255us/sample - loss: 0.0528 - val_loss: 0.0296
Epoch 2/2
28216/28216 [==============================] - 3s 124us/sample - loss: 0.0112 - val_loss: 0.0279
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Out

In [7]:
for model in results:
    model['left_rmse_mean'] = np.mean(model['left_validation_rmse'])
    model['right_rmse_mean'] = np.mean(model['right_validation_rmse'])

In [8]:
def mapper(x):
    out = {}
    out['window_size'] = x['model_config']['window_size']
    for key in x['model_config']['lstm'].keys():
        out['lstm_{}'.format(key)] = x['model_config']['lstm'][key]

    for key in x['model_config']['dense'].keys():
        out['dense_{}'.format(key)] = x['model_config']['dense'][key]

    for key in x['model_config']['optimizer'].keys():
        out['optim_{}'.format(key)] = x['model_config']['optimizer'][key]

    out['left_rmse_mean'] = x['left_rmse_mean']
    out['right_rmse_mean'] = x['right_rmse_mean']
    return out
  
input_df = list(map(mapper, results))

In [9]:
print(len(results))
df_results = pd.DataFrame(input_df)
display(df_results)

2


,window_size,lstm_units,lstm_activation,dense_activation,optim_loss,optim_optimizer,left_rmse_mean,right_rmse_mean
0,20,30,relu,tanh,mean_squared_error,rmsprop,2.217802,2.386229
1,20,30,tanh,tanh,mean_squared_error,rmsprop,2.155490,2.340311


In [10]:
# print(left_rmse)
# print(right_rmse)
# plt.figure(1)
# plt.plot(gp['left_true'])
# plt.plot(gp['left_pred'])
# plt.legend(['GT', 'Pred'])
# plt.show()

# Define list of hyperparameter options
# For each hyperparameter combination
#   Create model using hyperparameters
#   Train model
#   Store hyperparameters and validation loss
#   Store plot of training/validation loss over time

In [11]:
# n_epochs = len(model_hist.history['loss'])
# epochs = np.arange(1,n_epochs+1)
# plt.figure(1)
# plt.plot(epochs, model_hist.history['loss'])
# plt.plot(epochs, model_hist.history['val_loss'])
# plt.legend(['Training Loss', 'Validation Loss'])
# plt.show()